In [1]:
import json
import graphene

print(graphene.__version__)

def unorder(ordered_dict):
    return json.dumps(ordered_dict, indent=4)

def run(query, request):
    schema = graphene.Schema(query=query)
    result = schema.execute(request)
    if result.errors:
        raise Exception(result.errors)
    print( unorder(result.data) )

2.0.1


In [2]:
class User(graphene.ObjectType):
    name = graphene.String()
    best_friend = graphene.Field(lambda: User)
    friends = graphene.Field(graphene.List(lambda: User))
    
    best_friend_id = 100
    friend_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9] 

    def resolve_best_friend(self, info):
        return user_loader.load(self.best_friend_id)

    def resolve_friends(self, info):
        return user_loader.load_many(self.friend_ids)
    
class Query(graphene.ObjectType):
    
    me = graphene.Field(User)

    def resolve_me(self, info):
        return User(name='abcd')
    
def get_users(ids):
    print(f"get_users({ids}) SELECT * FROM table_user WHERE id IN {ids}")
    return [User(name=f"User{id}") for id in ids]

In [3]:
from promise import Promise
from promise.dataloader import DataLoader

class UserLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        print(f'self.batch_load_fn({keys})')
        return Promise.resolve(get_users(ids=keys))

In [4]:
user_loader = UserLoader()

print('====')
user_loader.load(1).then(lambda user: user_loader.load(user.best_friend_id))
print('====')
user_loader.load(2).then(lambda user: user_loader.load(user.best_friend_id))


====
self.batch_load_fn([1])
get_users([1]) SELECT * FROM table_user WHERE id IN [1]
self.batch_load_fn([100])
get_users([100]) SELECT * FROM table_user WHERE id IN [100]
====
self.batch_load_fn([2])
get_users([2]) SELECT * FROM table_user WHERE id IN [2]


<Promise at 0x102c49320 fulfilled with <__main__.User object at 0x102c49208>>

In [5]:
run(Query, """
{
  me {
    name
    bestFriend {
      name
    }
    friends {
      name
      bestFriend {
        name
      }
    }
  }
}
""")

self.batch_load_fn([3, 4, 5, 6, 7, 8, 9])
get_users([3, 4, 5, 6, 7, 8, 9]) SELECT * FROM table_user WHERE id IN [3, 4, 5, 6, 7, 8, 9]
{
    "me": {
        "name": "abcd",
        "bestFriend": {
            "name": "User100"
        },
        "friends": [
            {
                "name": "User1",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User2",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User3",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User4",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User5",
                "bestFriend": {
                    "name": "User100"
                }
       